### 4.6.5 2진수 계산에 대한 전체 코드

In [1]:
import numpy as np
# import cupy as np  # GPU를 사용하면 주석 해제
import matplotlib.pyplot as plt

# -- 각 설정값 --
n_time = 8  # # 시점의 수(2진수의 자릿수)
n_in = 2  # # 시점의 수(2진수의 자릿수)
n_mid = 32 # 은닉층 뉴런의 수
n_out = 1 # 출력층 뉴런의 수

eta = 0.01 # 학습률
n_learn = 5001 # 학습 횟수
interval = 500 # 경과 표시 간격

# -- 2진수 생성 --
max_num = 2**n_time  # 10진수 최댓값
binaries = np.zeros((max_num, n_time), dtype=int)  # 2진수를 저장하는 배열
for i in range(max_num):
    num10 = i  # 10진수 수
    for j in range(n_time):
        pow2 = 2 ** (n_time-1-j)  # 2의 거듭제곱
        binaries[i, j] = num10 // pow2
        num10 %= pow2
# print(binaries)

# -- RNN층 -- 
class SimpleRNNLayer:
    def __init__(self, n_upper, n):
        # 파라미터 초깃값
        self.w = np.random.randn(n_upper, n) / np.sqrt(n_upper)  # 자비에르 초기화 기반의 초깃값
        self.v = np.random.randn(n, n) / np.sqrt(n)  # 자비에르 초기화 기반의 초깃값
        self.b = np.zeros(n)

    def forward(self, x, y_prev):  # y_prev: 이전 시점의 출력
        u = np.dot(x, self.w) + np.dot(y_prev, self.v) + self.b
        self.y = np.tanh(u)  # 출력
    
    def backward(self, x, y, y_prev, grad_y):
        delta = grad_y * (1 - y**2)

        # 각 기울기
        self.grad_w += np.dot(x.T, delta)
        self.grad_v += np.dot(y_prev.T, delta)
        self.grad_b += np.sum(delta, axis=0)

        self.grad_x = np.dot(delta, self.w.T)
        self.grad_y_prev = np.dot(delta, self.v.T)

    def reset_sum_grad(self):
        self.grad_w = np.zeros_like(self.w)
        self.grad_v = np.zeros_like(self.v)
        self.grad_b = np.zeros_like(self.b)

    def update(self, eta):
        self.w -= eta * self.grad_w
        self.v -= eta * self.grad_v
        self.b -= eta * self.grad_b

# -- 전결합 출력층 --
class RNNOutputLayer:
    def __init__(self, n_upper, n):
        self.w = np.random.randn(n_upper, n) / np.sqrt(n_upper)  # 자비에르 초기화 기반의 초깃값
        self.b = np.zeros(n)

    def forward(self, x):
        self.x = x
        u = np.dot(x, self.w) + self.b
        self.y = 1/(1+np.exp(-u))   # 시그모이드 함수

    def backward(self, x, y, t):
        delta = (y-t) * y * (1-y)
        
        self.grad_w += np.dot(x.T, delta)
        self.grad_b += np.sum(delta, axis=0)
        self.grad_x = np.dot(delta, self.w.T) 

    def reset_sum_grad(self):
        self.grad_w = np.zeros_like(self.w)
        self.grad_b = np.zeros_like(self.b)

    def update(self, eta):
        self.w -= eta * self.grad_w
        self.b -= eta * self.grad_b

# -- 각 층의 초기화 --
rnn_layer = SimpleRNNLayer(n_in, n_mid)
output_layer = RNNOutputLayer(n_mid, n_out)

# -- 훈련 --
def train(x_mb, t_mb):
    # 각 출력값을 저장하는 배열
    y_rnn = np.zeros((len(x_mb), n_time+1, n_mid))
    y_out = np.zeros((len(x_mb), n_time, n_out))

    # 순전파
    y_prev = y_rnn[:, 0, :]
    for i in range(n_time):
        # RNN층
        x = x_mb[:, i, :]
        rnn_layer.forward(x, y_prev)
        y = rnn_layer.y
        y_rnn[:, i+1, :] = y
        y_prev = y

        # 출력층
        output_layer.forward(y)
        y_out[:, i, :] = output_layer.y

    # 역전파
    output_layer.reset_sum_grad()
    rnn_layer.reset_sum_grad()
    grad_y = 0
    for i in reversed(range(n_time)):
        # 출력층
        x = y_rnn[:, i+1, :]
        y = y_out[:, i, :]
        t = t_mb[:, i, :]
        output_layer.backward(x, y, t)
        grad_x_out = output_layer.grad_x

        # RNN층
        x = x_mb[:, i, :]
        y = y_rnn[:, i+1, :]
        y_prev = y_rnn[:, i, :]
        rnn_layer.backward(x, y, y_prev, grad_y+grad_x_out)
        grad_y = rnn_layer.grad_y_prev

    # 파라미터 갱신
    rnn_layer.update(eta)
    output_layer.update(eta)

    return y_out

# -- 오차 계산 --
def get_error(y, t):
    return 1.0/2.0*np.sum(np.square(y - t))  # 오차제곱합

for i in range(n_learn):
    # 무작위 10진수 --
    num1 = np.random.randint(max_num//2)
    num2 = np.random.randint(max_num//2)

    # -- 입력 데이터 생성 --
    x1= binaries[num1]
    x2= binaries[num2]
    x_in = np.zeros((1, n_time, n_in))
    x_in[0, :, 0] = x1
    x_in[0, :, 1] = x2
    x_in  = np.flip(x_in, axis=1)  # 작은 자릿수를 과거 시점으로

    # -- 정답 데이터 준비 --
    t = binaries[num1+num2]
    t_in = t.reshape(1, n_time, n_out)
    t_in = np.flip(t_in , axis=1)

    # -- 훈련 --
    y_out = train(x_in, t_in)
    y = np.flip(y_out, axis=1).reshape(-1)

    # -- 오차 구하기 --
    error = get_error(y_out, t_in)

    # -- 경과 표시 -- 
    if i%interval == 0:
        y2 = np.where(y<0.5, 0, 1)  # 2진수 결과
        y10 = 0  # 10진수 결과
        for j in range(len(y)):
            pow2 = 2 ** (n_time-1-j)  # 2의 거듭제곱
            y10 += y2[j] * pow2

        print("n_learn:", i)
        print("error:", error)
        print("output :", y2)
        print("correct:", t)

        c = "\(^_^)/ : " if (y2 == t).all() else "orz : "
        print(c + str(num1) + " + " + str(num2) + " = " + str(y10))
        print("-- -- -- -- -- -- -- -- -- -- -- -- -- -- --")

n_learn: 0
error: 1.153670208021627
output : [1 1 1 1 0 1 1 1]
correct: [1 0 1 0 0 0 0 1]
orz : 46 + 115 = 247
-- -- -- -- -- -- -- -- -- -- -- -- -- -- --
n_learn: 500
error: 1.0063991726943604
output : [0 0 1 0 1 1 0 0]
correct: [0 1 1 0 1 0 1 0]
orz : 100 + 6 = 44
-- -- -- -- -- -- -- -- -- -- -- -- -- -- --
n_learn: 1000
error: 0.8029258680156648
output : [1 1 1 1 1 1 1 0]
correct: [1 1 0 1 1 1 0 0]
orz : 109 + 111 = 254
-- -- -- -- -- -- -- -- -- -- -- -- -- -- --
n_learn: 1500
error: 0.7818524716336389
output : [0 1 1 1 0 1 1 0]
correct: [0 1 0 1 0 1 1 0]
orz : 34 + 52 = 118
-- -- -- -- -- -- -- -- -- -- -- -- -- -- --
n_learn: 2000
error: 0.7493708924813707
output : [1 1 1 0 1 1 0 0]
correct: [0 1 1 0 1 1 0 0]
orz : 19 + 89 = 236
-- -- -- -- -- -- -- -- -- -- -- -- -- -- --
n_learn: 2500
error: 0.5663644857676793
output : [0 0 1 1 1 1 1 1]
correct: [0 0 1 0 1 1 1 1]
orz : 22 + 25 = 63
-- -- -- -- -- -- -- -- -- -- -- -- -- -- --
n_learn: 3000
error: 0.4555234278887965
output : [